In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("HM5").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        getOrCreate()

23/12/30 15:04:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import numpy as np
import pandas as pd

In [3]:
read_data = pd.read_csv(r'google-data/web-Google.txt', sep='\t')
read_data

,FromNodeId,ToNodeId
0,0,11342
1,0,824020
2,0,867923
3,0,891835
4,11342,0
...,...,...
5105034,916425,547323
5105035,916425,604833
5105036,916425,632916
5105037,916425,637936


In [4]:
#Spark dataframe
pd.DataFrame.iteritems = pd.DataFrame.items
data = spark.createDataFrame(read_data)
data.show()
print((data.count(), len(data.columns)))

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/types.py:238: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):
23/12/30 15:07:13 WARN TaskSetManager: Stage 0 contains a task of very large size (14310 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------+
|FromNodeId|ToNodeId|
+----------+--------+
|         0|   11342|
|         0|  824020|
|         0|  867923|
|         0|  891835|
|     11342|       0|
|     11342|   27469|
|     11342|   38716|
|     11342|  309564|
|     11342|  322178|
|     11342|  387543|
|     11342|  427436|
|     11342|  538214|
|     11342|  638706|
|     11342|  645018|
|     11342|  835220|
|     11342|  856657|
|     11342|  867923|
|     11342|  891835|
|    824020|       0|
|    824020|   91807|
+----------+--------+
only showing top 20 rows



23/12/30 15:07:18 WARN TaskSetManager: Stage 1 contains a task of very large size (14310 KiB). The maximum recommended task size is 1000 KiB.


(5105039, 2)


#### List of web pages with the number of outlinks, sorted in descending order of the out-degrees.

In [5]:
import pyspark.sql.functions as sf

data.groupBy("FromNodeId")\
        .agg(sf.count('FromNodeId').alias('out-degree'))\
        .sort('out-degree', ascending=False)\
        .show(20, False)

23/12/30 15:07:25 WARN TaskSetManager: Stage 3 contains a task of very large size (14310 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+
|FromNodeId|out-degree|
+----------+----------+
|506742    |456       |
|203748    |372       |
|305229    |372       |
|768091    |330       |
|808643    |277       |
|412410    |268       |
|600479    |265       |
|376428    |258       |
|156950    |257       |
|885728    |256       |
|667584    |253       |
|685695    |248       |
|282140    |247       |
|598188    |245       |
|579314    |244       |
|411593    |231       |
|321091    |229       |
|838278    |225       |
|302733    |216       |
|915273    |213       |
+----------+----------+
only showing top 20 rows



#### InLink distribution of the top linked web pages, sorted in descending order of the in-degrees.

In [6]:
data.groupBy("ToNodeId")\
        .agg(sf.count('ToNodeId').alias('in-degree'))\
        .sort('in-degree', ascending=False)\
        .show(20, False)

23/12/30 15:07:38 WARN TaskSetManager: Stage 5 contains a task of very large size (14310 KiB). The maximum recommended task size is 1000 KiB.


+--------+---------+
|ToNodeId|in-degree|
+--------+---------+
|537039  |6326     |
|597621  |5354     |
|504140  |5271     |
|751384  |5182     |
|32163   |5097     |
|885605  |4847     |
|163075  |4731     |
|819223  |4620     |
|605856  |4550     |
|828963  |4484     |
|551829  |4220     |
|41909   |4219     |
|558791  |4206     |
|459074  |4187     |
|407610  |4180     |
|213432  |4084     |
|765334  |4015     |
|384666  |4010     |
|173976  |3988     |
|687325  |3956     |
+--------+---------+
only showing top 20 rows



#### Given a node v, please output the list of nodes that v points to, and the list of nodes that points to v.

In [7]:
while True:
    try:
        node = int(input("Please enter the node v:"))
    except ValueError:
        print("Your input is not valid.")
        continue
    if node < 0:
        continue
    else:
        break
print('list of nodes that v points to')
data.filter(data.FromNodeId == node)\
          .show(truncate=False)
print('list of nodes that points to v')
data.filter(data.ToNodeId == node)\
          .show(truncate=False)

Please enter the node v: 597621


list of nodes that v points to


23/12/30 15:08:19 WARN TaskSetManager: Stage 7 contains a task of very large size (14310 KiB). The maximum recommended task size is 1000 KiB.
23/12/30 15:08:19 WARN TaskSetManager: Stage 8 contains a task of very large size (14310 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------+
|FromNodeId|ToNodeId|
+----------+--------+
|597621    |2937    |
|597621    |89958   |
|597621    |107219  |
|597621    |137136  |
|597621    |163524  |
|597621    |178815  |
|597621    |190733  |
|597621    |206688  |
|597621    |397349  |
|597621    |518625  |
|597621    |564202  |
|597621    |564998  |
|597621    |654863  |
|597621    |660384  |
|597621    |718679  |
|597621    |727948  |
|597621    |729403  |
|597621    |769429  |
|597621    |783319  |
|597621    |857417  |
+----------+--------+
only showing top 20 rows

list of nodes that points to v
+----------+--------+
|FromNodeId|ToNodeId|
+----------+--------+
|2         |597621  |
|30957     |597621  |
|357310    |597621  |
|423174    |597621  |
|430119    |597621  |
|462435    |597621  |
|472889    |597621  |
|565424    |597621  |
|581609    |597621  |
|644135    |597621  |
|858904    |597621  |
|416       |597621  |
|199663    |597621  |
|46237     |597621  |
|219020    |597621  |
|567756    |597621 